In [1]:
## Carrega as bibliotecas necessárias

import requests # módulo para enviar requisições HTTP
import os # módulo para usar funcionalidades do sistema operacional
import pandas as pd # ferramentas de manipulação e análise de dados
import numpy as np # pacote para computação científica
import sys # para e funções específicas do sistema
import zipfile # módulo para trabalhar com arquivos zip
import io # ferramentas fundamentais para trabalhar com fluxos de dados
from bs4 import BeautifulSoup # biblioteca para extrair informação de arquivos HTML ou XML
import re # operações com expressões regulares
from itertools import groupby # agrupa valores de uma lista de acordo com o valor de uma função chave

if 'google.colab' in str(get_ipython()): # se o Google Colab estiver sendo usado, instala as bibliotecas necessárias
    from google.colab import files # módulo para fazer upload de arquivos a partir de sistema de arquivos local
    !pip install -q XlsxWriter # módulo para gerar e editar arquivos xlsx
else:  # caso contrário substitui o filtro padrão de mensagens de erro
    if not sys.warnoptions: # substitui o filtro padrão de mensagens de erro para ocultar os avisos de SSL
        import warnings
        warnings.simplefilter("ignore")

# Variável global para definir o uso o arquivo xml no lugar do arquivo txt: mudar para False se for fazer a busca
# somente com nos arquivos txt que estão disponíveis para todas as RPIs
# O xml é mais bem estruturado, o que pode reduzir os erros na extração, mas está disponível apenas a partir da
# RPI 2574, e a busca dentro dele é mais lenta
flag_xml = True
        
# A 2531 não encontrar o xml diretamente corrigir no código - CORRIGIDO MAS TESTAR
# Até a 2573 existe somente o arquivo TXT e não o xml - INCLUIR NA DOCUMENTAÇÂO
# O txt existe a partir da 1132 - INCLUIR NA DOCUMENTAÇÃO
# até a 1415 o ano da data de publicação tem dois digitos DD/MM/AA e não DD/MM/AAAA - CORRIGIDO MAS TESTAR

In [2]:
## Funções e classes para obter, fazer o download e gerar relatórios para a Revista de Propriedade Intelectual (RPI)

class api_inpi: # Classe para acessar e extrair informações patentárias da API do INPI
    
    def __init__(self, url): # construtor para inicializar as propriedades da classe
        self.url = url
    
    def arquivo_rpi(self,rpi): # verifica o status da RPI e extrai o arquivo xlm ou txt do pacote zip
        global flag_xml
        r = requests.head(self.url) # faz uma HEAD request e retorna o HTTP header com o código de status
        
        if r.status_code == 200: # se o código retornado for 200 procede com o download
            r = requests.get(url, stream=True) # solicita os dados do servidor
            file_zip = zipfile.ZipFile(io.BytesIO(r.content)) # faz o download do ZIP e extrai o conteúdo na memória
            f_names = file_zip.namelist() # lista o nome dos arquivos do arquivo ZIP da RPI
            
            for fileName in f_names: # para cada arquivo dentro lista de nomes
                if rpi < 2574 or flag_xml == False: # Até a RPI 2573, extraí o arquivo txt
                    if fileName.endswith('.txt') or fileName.endswith('.TXT'): # seleciona o arquivo txt
                        file_zip.extract(fileName) # extrai do pacote zip
                        unziped_file = fileName # e armazena o nome em uma variável
                if rpi >= 2574 and flag_xml == True: # A partir da RPI 2574, extraí o arquivo xml
                    if fileName.endswith('.xml') or fileName.endswith('.XML'): # seleciona o arquivo xml
                        file_zip.extract(fileName) # extrai do pacote zip
                        unziped_file = fileName # e armazena o nome em uma variável
                    
            return unziped_file # retorna o nome do arquivo
        
        if r.status_code == 302: # se o código for 302, o arquivo ainda não está disponível
            print("O Arquivo da RPI ainda não está disponível, tente mais tarde.")
            
    # Procura os pareceres de patentes pelo nome do titular
    def busca_titular(self,rpi,org,df):
        global flag_xml
        f_rpi = self.arquivo_rpi(rpi) # obtêm o arquivo xml da rpi
        
        if rpi < 2574 or flag_xml == False: # Até a RPI 2573 ou parse o TXT
            with open(f_rpi, 'rt', encoding = "ISO-8859-1") as f: # abre o arquivo e codifica em ISO-8859-1
                file_data = f.readlines() # armazena o conteúdo do arquivo em uma variável
            
            file_data = [re.sub(r' +', ' ', i) for i in file_data] # remove os espaços em branco contínuos do documento
            file_data = [re.sub(r' $', '', i) for i in file_data] # remove o espaço em branco no final do elemento da  lista, se houver
            file_data = [re.sub(r' \n', '', i) for i in file_data] # remove ocorrências de \n dentro dos elementos da lista
            file_data = [re.sub(r'\|\n', '', i) for i in file_data] # remove ocorrências de \n dentro dos elementos da lista
            file_data = [re.sub(r'\n', '', i) for i in file_data] # remove ocorrências de \n dentro dos elementos da lista
            
            file_data = [re.sub(r'(?i)\(cd\) ', '(cd)', i) for i in file_data] # normaliza o marcador de código do serviço em lower case
            file_data = [re.sub(r'(?i)\(co\) ', '(co)', i) for i in file_data] # normaliza o marcador de comentário em lower case
            
            if rpi <= 1415: # neste caso, o ano da publicação tem o formato DD/MM/AA
                data_revista = str(file_data[0][-8:-3]) + "/19" + str(file_data[0][-2:]) # incluí o número 19 na frente do ano          
            if rpi > 1415:  # neste caso, o ano da publicação tem o formato DD/MM/AAAA
                data_revista = file_data[0][-10:] # extrai da data de publicação da RPI
            
            # separa o inid do código de despacho, delimitando os subgrupos do arquivo
            grouped_data = []
            for i in file_data:
                grouped_data+=i.split("(cd)")
            
            del file_data # apaga a variável temporária com os dados do arquivo da rpi
            
            # Divide a listagem da RPI em subgrupos a partir da marca deixada pelo código de despacho
            grouped_data = [list(j) for i, j in groupby(grouped_data, lambda x:x == "") if not i]
            
            # Busca pela organização
            match = []
            for x, content_line in enumerate(grouped_data): # faz a busca nas listas de patentes
                for y, content_column in enumerate(content_line): # faz a busca dentro de cada lista de despachos
                    if org.lower() in content_column.lower():
                        for w, reg in enumerate(grouped_data[x]): # faz a busca novamente na lista de patentes
                            if "(11) " in reg: # mas buscando o número da patente
                                match.append(reg[5:]) # para incluir em uma lista de índices
                            if "(11) " not in reg: # caso não encontre o INID 11, número da patente
                                if "(21) " in reg: # busca pelo INID 21, número do pedido
                                    match.append(reg[5:]) # para incluir em uma lista de índices
                                    
            # Busca pela ocorrência do número de patentes na lista de índices
            for z, reg in enumerate(match): # navega pela lista de índices
                for x, content_line in enumerate(grouped_data): # faz a busca nas listas de patentes
                    for y, content_column in enumerate(content_line): # faz a busca dentro de cada lista de despachos
                        if match[z] in content_column: # se a proteção está dentro da lista de despachos
                            for w, data_dep in enumerate(grouped_data[x]): # faz a busca novamente na lista de patentes 
                                if "(22) " in data_dep: # buscando pela data de depósito
                                    data_deposito = data_dep[5:]                                    
                                    df.loc[len(df)] = [rpi,data_revista,reg,data_deposito]
                                    
        if rpi >= 2574 and flag_xml == True: # Faz a busca no arquivo xml, disponível a partir da RPI 2574
            infile = open(f_rpi,'r') # abre o arquivo da rpi
            file_data = infile.read() # armazena o conteúdo do arquivo em uma variável
            file_data = re.sub(' +',' ',file_data) # substitui espaços múltiplos por espaçamentos simples
            soup = BeautifulSoup(file_data, 'xml') # passa os dados armazenados para o analisador do beautifulsoup
            
            data_revista = soup.find('revista').attrs.get('dataPublicacao') # extrai a data de publicação
        
            # procura todas as ocorrências do nome do titular parece no arquivo xml
            org_soup = soup.find_all('nome-completo', text=lambda x: x is not None and org in x.casefold())
            despacho_len = len(org_soup) # conta o número de registros encontrados
                
            for i in range(despacho_len): # busca no arquivo xml para extrair as informações onde o titular foi encontrado
                despacho_soup = org_soup[i].find_parent('despacho') # para o i-th elemento, encontra a tag pai <despacho>
            
                # para o i-th elemento, busca quantas vezes o número da proteção foi encontrado
                registro_soup = soup.find_all('numero', text=lambda x: despacho_soup.find('numero').text in x)
            
                registro_len = len(registro_soup) # conta quantas vezes o número da proteção foi repetido
            
                # busca as informações para todas as vezes em que o número da proteção foi repetido
                for j in range(registro_len):
                    registros = registro_soup[j].find_parent('despacho')
                
                    registro = registros.find('numero') # busca pelo número da proteção
                    if registro != None:
                        registro = registro.text
                    
                    # define os códigos de busca padronizados, para fazer a busca no servidor web.
                    # se o número de proteção começar com BR, remove BR, os espaços, o digito identificador e o hífen
                    # para outras proteções, remove apenas o digito identificador e o hífen
                    cod_busca = registro[:-2].replace(' ', '').replace('BR', '')
        
                    cod_despacho = registros.find('codigo') # busca pelo código de despacho
                    if cod_despacho != None:
                        cod_despacho = cod_despacho.text
                    
                    titulo = registros.find('titulo', inid="54") # busca pelo título da patente
                    if titulo != None:
                        titulo = titulo.text
                                
                    kindcode = registros.find('numero').attrs.get('kindcode') # busca pelo kind code
                    
                    data_deposito = registros.find('data-deposito') # busca pela data de depósito
                    if data_deposito != None:
                        data_deposito = data_deposito.text
                
                    # busca pelas informações de classificação internacional 
                    ci_len = len(registros.find_all('classificacao-internacional'))
                    ci = ""
                    if ci_len > 0:
                        for k in range(ci_len): # para o k-th elemento
                            # armazena as informações de classificação internacional no formato
                            # código de classificação internacional (ano) | ...
                            ci_ano = registros.find_all('classificacao-internacional')[k].text + " (" + registros.find_all('classificacao-internacional')[k].attrs.get('ano') + ")"
                            ci = ci + ci_ano + " | "
                    ci = ci[:-3]
                
                    # busca pelas informações de titularidade 
                    titulares_len = len(registros.find_all('titular'))
                    titulares = ""
                    gestao = "Sim"
                    if titulares_len > 0:
                        for k in range(titulares_len): # para o k-th elemento
                            titular = registros.find_all('titular')[k].find('nome-completo').text
                            if k == 0: # verifica o responsável pelo depósito da patente
                                # se o primeiro titular não for a organização procurada,
                                # define a variável gestão como Não
                                if org.lower() not in titular.lower():
                                    gestao = "Não"                            
                            # armazena as informações de titularidade no formato
                            # titular 1 | titular 2 | ...
                            titulares = titulares + titular + " | "
                    titulares = titulares[:-3]
                
                    # busca pelo nome dos inventores    
                    inventor_len = len(registros.find_all('inventor'))
                    inventores = ""
                    if inventor_len > 0:
                        for k in range(inventor_len): # para o k-th elemento
                            # armazena o nome dos inventores no formato
                            # inventor 1 | inventor 2 | ...
                            inventor = registros.find_all('inventor')[k].find('nome-completo').text
                            inventores = inventores + inventor + " | "
                    inventores = inventores[:-3]
                
                    # armazena a informação extraída na i-th linha do dataframe
                    df.loc[len(df)] = [rpi, data_revista,registro, kindcode, cod_despacho,titulo,gestao, titulares, inventores, data_deposito, ci,cod_busca]
        
            # define a coluna rpi do dataframe como tipo inteiro para evitar conflitos
            df['rpi'] = df['rpi'].astype('int')
            
            # SÓ HABILITAR DEPOIS QUE IMPLEMENTAR PARA O TXT
            # define a cluna cod_busca como tipo string para evitar conflitos
            #df['cod_busca'] = df['cod_busca'].astype('str')
            
            # SÓ HABILITAR DEPOIS QUE IMPLEMENTAR PARA O TXT
            # define a cluna cod_despacho como tipo string para evitar conflitos
            #df['cod_despacho'] = df['cod_despacho'].astype('str')
        
        os.remove(f_rpi) # apaga o arquivo da rpi

# Generate a report in xlsx format with the information parsed from the RPI
#     Use: excel_report(source dataframe,file name,inside file sheet name)
def excel_report(xls_df,xls_name,sheet_name):
    writer = pd.ExcelWriter(xls_name + ".xlsx", engine='xlsxwriter', date_format='dd/mm/yyyy') # create a pandas excel writer using XlsxWriter
    xls_df.to_excel(writer, sheet_name=sheet_name, index=False) # convert the dataframe to an XlsxWriter Excel object
    workbook  = writer.book # get the xlsxwriter workbook object
    worksheet = writer.sheets[sheet_name] # get the xlsxwriter worksheet object
    
    for i, col in enumerate(xls_df.columns): # iterate through each column and set the width to the max length
        column_len = xls_df[col].astype(str).str.len().max() # find length of column i
        column_len = max(column_len, len(col)) + 2 # setting the length if the column header is larger than the max column value length
        worksheet.set_column(i, i, column_len) # set the column length
        
    writer.save() # close the Pandas Excel writer and output the Excel file
    print("RPI Report concluded!")

print("Bibliotecas carregadas.")

Bibliotecas carregadas.


In [3]:
## RPI selection

rpi_i =  input("Digite o número da primeira RPI:\n")
rpi_i = int(rpi_i)
rpi_f =  input("Digite o número da última RPI:\n(Caso seja somente uma edição, aperte ENTER)\n") or rpi_i
rpi_f = int(rpi_f)

Digite o número da primeira RPI:
1132
Digite o número da última RPI:
(Caso seja somente uma edição, aperte ENTER)
1416


In [4]:
## Make the search in patent section of the RPI

# Set the columns and create the dataframe to store information
#column_names = ['rpi', 'data de publicação','registro','kind code','cod_despacho','título','gestão','titulares','inventores','data de deposito','classificação internacional','cod_busca'] # cria as colunas do dataframe
#patentes_df = pd.DataFrame(columns=column_names)

# teste para o TXT, apagar depois que consolidar os dois tipos de arquivo
column_names = ['rpi','data_rpi','registro','data_de_deposito'] # cria as colunas do dataframe
patentes_df = pd.DataFrame(columns=column_names)

rpi = rpi_i # set rpi counter as rpi_i
for i in range(rpi_f-rpi_i+1): # for the i-th edition from the rpi
    url = "http://revistas.inpi.gov.br/txt/P" + str(rpi) + ".zip" # set the url to make the search
    api_inpi(url).busca_titular(rpi,'universidade estadual de campinas',patentes_df) # make the search
    print("RPI", rpi, "concluída.")
    rpi+=1 # ident the rpi counter

if rpi_f > rpi_i: # if the last rpi is greather than firts
    f_name = "rpi_patentes_" + str(rpi_i) + "-" + str(rpi_f) # set the xlsx filename
    s_name = str(rpi_i) + " - " + str(rpi_f) # set the sheet filename
if rpi_f <= rpi_i: # else 
    f_name = "rpi_patentes_" + str(rpi_i)  # set the xlsx filename
    s_name = str(rpi_i)  # set the sheet filename

excel_report(patentes_df,f_name,s_name) # generate the xlsx report with extracted information

RPI 1132 concluída.
RPI 1133 concluída.
RPI 1134 concluída.
RPI 1135 concluída.
RPI 1136 concluída.
RPI 1137 concluída.
RPI 1138 concluída.
RPI 1139 concluída.
RPI 1140 concluída.
RPI 1141 concluída.
RPI 1142 concluída.
RPI 1143 concluída.
RPI 1144 concluída.
RPI 1145 concluída.
RPI 1146 concluída.
RPI 1147 concluída.
RPI 1148 concluída.
RPI 1149 concluída.
RPI 1150 concluída.
RPI 1151 concluída.
RPI 1152 concluída.
RPI 1153 concluída.
RPI 1154 concluída.
RPI 1155 concluída.
RPI 1156 concluída.
RPI 1157 concluída.
RPI 1158 concluída.
RPI 1159 concluída.
RPI 1160 concluída.
RPI 1161 concluída.
RPI 1162 concluída.
RPI 1163 concluída.
RPI 1164 concluída.
RPI 1165 concluída.
RPI 1166 concluída.
RPI 1167 concluída.
RPI 1168 concluída.
RPI 1169 concluída.
RPI 1170 concluída.
RPI 1171 concluída.
RPI 1172 concluída.
RPI 1173 concluída.
RPI 1174 concluída.
RPI 1175 concluída.
RPI 1176 concluída.
RPI 1177 concluída.
RPI 1178 concluída.
RPI 1179 concluída.
RPI 1180 concluída.
RPI 1181 concluída.


In [25]:
patente = "BR 11 2014 02740-6 CBA"
if len(patente) > 18:
    kind_code = patente[-len(patente)+19:]
    patente = patente[:-len(patente)+18]
print(kind_code, patente)

CBA BR 11 2014 02740-6


In [22]:
patente[:-len(patente)+18]

''

In [ ]:
## Download the files from the web server

# creates the columns with the file names and the folder names to download
# in this example we use this pattern to folders, with three levels
#     PareceresRPI/BR 10 2014 021620-0/RPI2589_180820
# and this pattern to files
#     PI 1104516-7_Despacho 7.1_180820_A.pdf
patentes_df['f_name'] = patentes_df['registro'] + "_Despacho " + patentes_df['cod_despacho'] + "_" + patentes_df['data de publicação'].str[0:2] + patentes_df['data de publicação'].str[3:5] + patentes_df['data de publicação'].str[8:11]
patentes_df['folder_name'] = "PareceresRPI/" + patentes_df['registro'] + "/RPI" + patentes_df['rpi'].astype(str) + "_" + patentes_df['data de publicação'].str[0:2] + patentes_df['data de publicação'].str[3:5] + patentes_df['data de publicação'].str[8:11]
patentes_df.drop(patentes_df.columns.difference(['rpi','registro','cod_despacho','inpi_name','f_name','folder_name','cod_busca']), axis=1, inplace=True)

# set te columns and create the dataframe to store information from INPI web server
# for all selected editions
column_names = ['inpi_name','rpi','cod_busca']
df_inpi = pd.DataFrame(columns=column_names)

rpi = rpi_i # set rpi counter as rpi_i
for i in range(rpi_f-rpi_i+1): # for the i-th edition from the rpi
    url = "https://parecer.inpi.gov.br/arquivos/RPI/" + str(rpi) # set the url to make the search
    
    if requests.get(url, verify=False).status_code != 200: # check if the files are avaiabel at web server
        print("There are no files on the server for this edition.")
        sys.exit()
    
    # create a temporary dataframe to store the extract information from i-th edition
    df_temp = pd.read_html(requests.get(url, verify=False).content, encoding='utf-8', header=0)[0].iloc[2:-1]
    
    df_temp.drop(df_temp.columns.difference(['Name']), axis=1, inplace=True) # drop all columns, except Name
    df_temp.columns = ['inpi_name'] # rename the column
    df_temp.loc[:,'rpi'] = rpi # create the columns with i-th RPI number for all records
    df_temp['cod_busca'] = df_temp['inpi_name'].str[3:-11] # create the column with the search code
    df_inpi = df_inpi.append(df_temp) # append the records for the i-th rpi at the end of INPI dataframe 
    del df_temp # delete temporary dataframe
    rpi+=1 # ident the rpi number
    
df_inpi = df_inpi.reset_index(drop=True) # reset the index from rpi dataframe

# merge the datframes with the information extracted from RPI and INPI web server
patentes_df = pd.merge(patentes_df, df_inpi, left_on=['rpi','cod_busca'],right_on=['rpi','cod_busca'],how='left')    
patentes_df.drop(['cod_busca'], axis=1, inplace=True) # after this, drop the column with the search code

# remove the records that are not in both dataframes
patentes_df.dropna(subset = ['registro'], inplace=True) # remove the lines where pi is NaN
patentes_df.dropna(subset = ['inpi_name'], inplace=True) # remove the lines where inpi_name is NaN

patentes_df['download'] = True # create a column and set dowload as True

# cases where a protection has two orders in the same magazine
# it is necessary to keep only the documents for one of the dispatch numbers
# so set dowload as False
patentes_df.loc[patentes_df.cod_despacho == '15.11', 'download'] = False # the documents are duplicated between 15.11 and 6.22
patentes_df.loc[patentes_df.cod_despacho == '8.7', 'download'] = False # the documents are duplicated between 7.5 and 8.7
# the patent certificate is not on the same server as the patent reports
# and must be downloaded directly from the INPI website
patentes_df.loc[patentes_df.cod_despacho == '16.1', 'download'] = False # set download as False
patentes_df.loc[patentes_df['download'] == False,'download'] = np.nan # change False to NaN at download
patentes_df.dropna(subset = ['download'], inplace=True) # remove the lines where download is NaN  
patentes_df.drop(['download'], axis=1, inplace=True) # remove the download column

# adding count flags to reports when there is more than one document
patentes_df['dupl'] = patentes_df['registro'].duplicated(keep=False) # create dupl column
patentes_df['flag'] = patentes_df.groupby('registro').cumcount() # group by register number and count
patentes_df.loc[patentes_df['dupl'] == False, 'flag'] = "" # if there only one document, the flag column is empty
# in this example, we changed the numeric counters to alphabetic counters using a dict
dict_letters = {'' : '', 0 : '_A',1 : '_B', 2 : '_C', 3 : '_D', 4 : '_E', 5 : '_F', 6 : '_G', 7 : '_H', 8 : '_I', 9 : '_J'}
patentes_df['flag']= patentes_df['flag'].map(dict_letters)  # change the counters
patentes_df.drop(['dupl'], axis=1, inplace=True) # remove the column dupl
patentes_df['f_name'] = patentes_df['f_name'] + patentes_df['flag'] + '.pdf' # add the count flags and pdf extension to filename
patentes_df.drop(['flag'], axis=1, inplace=True) # remove the column flag

i = 1 # start position in the dataframe to perform the download of the files
# for the i-th elementh in the dataframe
for folder_name,inpi_name,f_name,rpi in zip(patentes_df['folder_name'],patentes_df['inpi_name'],patentes_df['f_name'],patentes_df['rpi']):
    if not os.path.exists(folder_name): # if the download folder not exist
        os.makedirs(folder_name) # create the folder
    url_cam = "http://parecer.inpi.gov.br/download.php?cam=arquivos/RPI/" + str(rpi) + "/" + inpi_name # set the url to download
    f_request = requests.get(url_cam, verify = False)  # request data from the resource
    path = folder_name + "/" + f_name # set the path to donwload the file
    f = open(path, 'wb').write(f_request.content) # make the download
    i+=1 # ident the position counter

print("Download completed!")

In [5]:
file = "datas_deposito1.xlsx"

df = pd.read_excel(file)
df = df.drop_duplicates(subset=['registro'])
excel_report(df,"datas_deposito","datas_deposito") # generate the xlsx report with extracted information

RPI Report concluded!


In [123]:
teste = ['(Cd) ', '(cD) ','(CD) ','(DC) ']

teste = [re.sub(r'(?i)\(cd\) ', '(cd)', file) for file in teste]

print(type(teste))
teste

#data_revista = str(teste[-8:-3]) + "/19" + str(teste[-2:])
#data_revista

<class 'list'>


['(cd)', '(cd)', '(cd)', '(DC) ']

In [103]:
file_lst = ['cats1.fa', 'cats2.fa', 'dog1.fa', 'dog2.fa']
file_lst_trimmed =[]

file_lst_trimmed = [re.sub(r'\d\.fa$', '', file) for file in file_lst]
type(file_lst_trimmed)

list